In [1]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


In [2]:
# read in known applicant files, dedupe and try to merge with applicants file
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name, long_form_date, 
                                    correct_mispellings, is_foreign_med_school, clean_med_school)

from dev import (
    APP_DATA_DIR, SUM_STAT_DIR, ATT_DATA_DIR, CARD_DATA_DIR, CORRECTIONS_DIR, AWARDS_KEYWORDS, NAME_COLS, RAW_NAME_COLS, 
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, PICKLE_DIR)
from merging_functions import *

OUTPUT_CSV = False 

PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


time: 229 ms


In [3]:
# id column that links back to raw applicant data file
RAW_CARD_ID = 'raw_uuid'

# column where the raw id information is stored
RAW_INDEX_IDS = 'raw_card_ids'

# try to get one id per unique applicant in the dataset
PERSON_ID = 'person_uuid'
# id per deduped application-person - if someone applied multiple times, they will have multiple ids
PERSON_APPLICATION_ID = 'person_app_uuid' 
NIH_ID = 'dno'

APPLICANT_SUFFIX = '_ap'
ATTENDEE_SUFFIX = '_at'

%load_ext autoreload
%autoreload 2

%reload_ext autoreload

time: 34.8 ms


In [4]:
apps_filename = 'index_cards_deduped_fuzzy.csv'
# apps_filename = 'person_application_date_wide.csv'

NIH_filename = 'unique_attendees.csv'
# NIH_filename = 'NIH_attendee_deduped_raw.csv'

time: 12.8 ms


In [5]:
NAME_COLS = ['clean_first_name', 'clean_middle_name', 'clean_last_name'] 

MED_TRAINING_COLS = ['res_dates', 'intern_dates', 'residency_hospital', 'internship_hospital', 'medical_school', 'residency']


time: 34.5 ms


In [6]:
# import NIH raw data set
NIH_raw = pd.read_csv(os.path.join(ATT_DATA_DIR, NIH_filename)).drop_duplicates('dno')

time: 75.3 ms


In [7]:
# drop from the data set all people with eod years > 1980
# NIH = NIH_raw.loc[NIH_raw.eod_year<1980, :] 
NIH = NIH_raw
del NIH['medical_school']

time: 12.5 ms


In [8]:
NIH['clean_medical_school'] = NIH.med_school.str.upper().apply(clean_med_school)

UNIVERSITY OF ATHENS MEDICAL SCHOOL IN ATHENS GREECE
UNIVERSITY OF SAINT ANTOINE SCHOOL OF MEDICINE PARIS FRANCE
time: 3.5 s


In [9]:
NIH.loc[NIH.clean_last_name=='LEVY', NAME_COLS+['med_school', 'clean_medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,med_school,clean_medical_school
176,ARNOLD,GLENN,LEVY,The George Washington University,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE
201,ARTHUR,LOUIS,LEVY,Albert Einstein College of Medicine,ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA...
291,BERNARD,S,LEVY,Duke University School of Medicine,DUKE MEDICAL SCHOOL
1578,JAY,A,LEVY,College of Physicians and Surgeons,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...
3176,ROBERT,ISAAC,LEVY,Yale,YALE UNIVERSITY SCHOOL OF MEDICINE
3370,RONALD,ISAAC,LEVY,Yale,YALE UNIVERSITY SCHOOL OF MEDICINE
3642,STUART,BLANK,LEVY,"University of Pennsylvania, School Of Medicine",UNIVERSITY OF PENNSYLVANIA SCHOOL OF MEDICINE


time: 31.8 ms


In [10]:
# import cleaned, deduped applicant data set in wide form (multiple app dates as columns)
apps = pd.read_csv(os.path.join(APP_DATA_DIR, apps_filename))


time: 113 ms


In [11]:
apps.loc[pd.isnull(apps.application_year), NAME_COLS+['year_accepted', 'application_date']]

,clean_first_name,clean_middle_name,clean_last_name,year_accepted,application_date
282,BARRY,SPENCER,COLLER,1972.0,2016-04-11
620,DAVID,CAMERON,MCCHURE,NaN,1972-02-07
1998,JOSEPH,GEORGE,PERPICH,1968,1941-07-22


time: 26.6 ms


In [12]:
apps.loc[apps.clean_first_name=='HOWARD UNIVERSITY COLLEGE OF MEDICINE', 'clean_first_name'] = 'HOWARD'
apps.loc[apps.clean_middle_name=='HOWARD UNIVERSITY COLLEGE OF MEDICINE', 'clean_middle_name'] = 'HOWARD'
apps.loc[apps.clean_first_name=='STANFORD UNIVERSITY', 'clean_first_name'] = 'STANFORD'

NIH.loc[NIH.clean_first_name=='HOWARD UNIVERSITY COLLEGE OF MEDICINE', 'clean_first_name']
NIH.loc[NIH.clean_first_name=='STANFORD UNIVERSITY', 'clean_first_name'] = 'STANFORD'

apps.loc[apps.clean_last_name=='HARRIN', NAME_COLS+['medical_school', 'application_year']]
apps.loc[apps.clean_last_name=='MCCLURE MCCHURE', 'clean_last_name'] = 'MCCLURE'
apps.loc[apps.clean_last_name=='MCCHURE', 'clean_last_name'] = 'MCCLURE'
apps.loc[apps.clean_last_name=='MCCLURE', 'application_year'] = 1972

time: 28.9 ms


In [13]:
apps.loc[apps.clean_last_name=='COLLER', 'application_year'] = 1972

apps.loc[apps.clean_last_name=='PERPICH', 'application_year'] = 1967
apps.loc[apps.clean_last_name=='PERPICH', 'clean_first_name'] = 'JOSEPH'

apps = apps.loc[~(apps.clean_last_name=='MCCHURE')]

time: 21.5 ms


In [14]:
NIH.loc[NIH.clean_last_name=='PERPICH', 'eod_year']

1948    1968.0
Name: eod_year, dtype: float64

time: 21.9 ms


In [15]:
apps.loc[:, 'res_dates'] = apps['residency_year(s)'].apply(long_form_date)

apps.loc[:, 'intern_dates'] = apps['internship_year(s)'].apply(long_form_date)

"Open"
Str. Medicine
Medicine
time: 60.6 ms


In [16]:
NIH = NIH.rename(columns={'res_dtes':'res_dates', 'intern_dte':'intern_dates', 'intern_hos': 'internship_hospital', 
                         'res_hosp':'residency_hospital', 'clean_medical_school': 'medical_school'})
# sorted(NIH.columns)

time: 16.6 ms


In [17]:
apps2 = apps.rename(
    columns={'residency_type': 'residency', 'internship_hospital_1': 'internship_hospital'})

string_med_cols = ['medical_school', 'residency_hospital', 'internship_hospital', 'residency']

# apply string cleaning function to each of the string medical info columns
apps2.loc[:, string_med_cols] = apps2[string_med_cols].applymap(clean_names)

time: 189 ms


In [18]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

apps2.loc[:, 'clean_last_name'] = apps2.clean_last_name.apply(correct_name_mispellings_fnc)
apps2.loc[:, 'clean_first_name'] = apps2.clean_first_name.apply(correct_name_mispellings_fnc)

time: 75.9 ms


In [19]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        sel = df.loc[df[selection_type]==selection_value, t]
        print sel.shape[0]
        if sel.shape[0] > 1:
            print "WARNING"
        df.loc[df[selection_type]==selection_value, t] = v

time: 13.9 ms


In [20]:

apps2.loc[(apps2.clean_last_name=='LIBOW') & (apps2.clean_middle_name=='S'), 'clean_first_name'] = 'LESLIE'

change_names(
    apps2, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(
    apps2, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    apps2, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    apps2, 'clean_last_name', 'BOYD', ['clean_first_name', 'clean_middle_name'], ['MICHAEL', 'RAY'])
change_names(
    apps2, 'clean_last_name', 'CHAPMAN', 
    ['clean_first_name', 'clean_middle_name', 'medical_school'], ['STANLEY', 'WILLETS', 'ROCHESTER'])

clean_first_name BRUCE
1
clean_middle_name WILCOX
1
clean_first_name MARC
1
clean_middle_name I
1
clean_first_name SIDNEY
1
clean_middle_name CHARLES
1
medical_school WAKE_FOREST
1
clean_first_name MICHAEL
1
clean_middle_name RAY
1
clean_first_name STANLEY
1
clean_middle_name WILLETS
1
medical_school ROCHESTER
1
time: 97.6 ms


In [21]:
change_names(apps2, 'clean_last_name', 'DANFORTH', ['clean_first_name'], ['DAVID'])
change_names(apps2, 'clean_last_name', 'HUNT', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'D'])
change_names(apps2, 'clean_last_name', 'KARK', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'ADRIAN'])
change_names(apps2, 'clean_last_name', 'KEBABIAN', ['clean_first_name', 'clean_middle_name'], ['JOHN', 'WILLIS'])
change_names(apps2, 'clean_last_name', 'KNOPF', ['clean_first_name', 'clean_middle_name'], ['HARRY', 'LOUIS'])
change_names(apps2, 'clean_last_name', 'KROLIKOWSKI', ['clean_first_name', 'clean_middle_name'], ['FRANCIS', 'JOHN'])
change_names(apps2, 'clean_last_name', 'KASTL', ['clean_first_name', 'clean_middle_name'], ['DAVID', 'GENE'])
change_names(apps2, 'clean_last_name', 'KLAVEMAN', ['clean_last_name'], ['KLAEVEMAN'])
change_names(apps2, 'clean_last_name', 'MATHEW', ['clean_last_name'], ['MATTHEW'])

apps2.loc[apps2.clean_last_name=='CHESEBRO', ['clean_first_name']] = 'BRUCE'
apps2.loc[apps2.clean_last_name=='CHESEBRO', ['clean_middle_name']] = 'WILCOX'
apps2.loc[(apps2.clean_last_name=='HEALY') & (apps2.medical_school=='USC KECK SCHOOL OF MEDICINE'), ['clean_first_name']] = 'MARK'
apps2.loc[(apps2.clean_last_name=='HEALY') & (apps2.medical_school=='USC KECK SCHOOL OF MEDICINE'), ['clean_middle_name']] = 'H'

apps2.loc[apps2.clean_last_name=='LENN', 'clean_first_name'] = 'NICHOLAS'
apps2.loc[apps2.clean_last_name=='BRESLOW', 'clean_first_name'] = 'JAN'
apps2.loc[apps2.clean_last_name=='BRESLOW', 'clean_middle_name'] = 'LESLIE'

apps2.loc[(apps2.clean_last_name=='NADLER') & (pd.isnull(apps2.clean_first_name)), 'clean_first_name'] = 'LEE'
apps2.loc[(apps2.clean_last_name=='NADLER') & (apps2.clean_first_name=='LEE'), 'clean_middle_name'] = 'MARSHALL'
apps2.loc[(pd.isnull(apps2.clean_first_name)) & (apps2.clean_last_name=='ROSEN'), 'clean_first_name'] = 'HENRY'

apps2.loc[(apps2.clean_last_name=='NEELON'), 'clean_first_name'] = 'FRANCIS'
apps2.loc[(apps2.clean_last_name=='NEELON') , 'clean_middle_name'] = 'ALBERT'

apps2.loc[(apps2.clean_last_name=='NICHOLAS') , 'clean_first_name'] = 'JOHN'

clean_first_name DAVID
1
clean_first_name ROBERT
0
clean_middle_name D
0
clean_first_name ROBERT
1
clean_middle_name ADRIAN
1
clean_first_name JOHN
0
clean_middle_name WILLIS
0
clean_first_name HARRY
1
clean_middle_name LOUIS
1
clean_first_name FRANCIS
1
clean_middle_name JOHN
1
clean_first_name DAVID
1
clean_middle_name GENE
1
clean_last_name KLAEVEMAN
0
clean_last_name MATTHEW
0
time: 102 ms


In [22]:
# there are a bunch of duplicates in apps, where application year is the same, but first name is missing
name_dups = apps2.loc[
    apps2.duplicated(
        ['clean_last_name', 'medical_school', 'application_year'], keep=False), NAME_COLS+[PERSON_ID, 'medical_school', 'application_year']]

time: 38.8 ms


In [23]:
to_delete_ids = name_dups.loc[(pd.isnull(name_dups.clean_middle_name)) & (pd.isnull(name_dups.clean_first_name))
                             & pd.isnull(name_dups.medical_school), PERSON_ID]

time: 78.2 ms


In [24]:
apps4 = apps2.loc[~apps2[PERSON_ID].isin(to_delete_ids), :].sort_values(
    NAME_COLS+['medical_school'], ascending=False).drop_duplicates(NAME_COLS+['medical_school'])

time: 73.1 ms


In [25]:
print apps4.shape
print apps2.shape

(4115, 86)
(4116, 86)
time: 52 ms


In [26]:
NIH.loc[(NIH.clean_last_name=='ALEXANDER') & (NIH.clean_first_name=='JOHN'), 'clean_middle_name'] = 'CHARLES'

time: 46 ms


In [27]:
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_middle_name'] = 'ARTHUR'
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_first_name'] = 'EDWARD'
NIH.loc[(
        NIH.clean_last_name=='LEBOWITZ') & (
            NIH.clean_first_name=='EDWARD'), 'medical_school'] = 'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA'

time: 28.7 ms


In [28]:
NIH = NIH.loc[NIH.clean_first_name!='GERALDINE']

time: 58.7 ms


In [29]:
# read in manual matches
man = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man['medical_school'] = man.dno_medical_school.apply(clean_med_school)

time: 158 ms


In [30]:
t = pd.merge(left=apps4, right=man, on=['clean_last_name', 'clean_middle_name'], how='inner', suffixes=['_x', '_y'])
print t.shape

t['sim'] = t[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

t['clean_first_name'] = t['clean_first_name_y']

t_1 = t.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')

# t[NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]
t_1.loc[t_1.duplicated('dno', keep=False), NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]

(112, 92)


,clean_first_name,clean_middle_name,clean_last_name,dno,medical_school_x,medical_school_y,sim


time: 49.8 ms


In [31]:
apps4.loc[(apps4.clean_last_name=='COHEN') & (apps4.clean_first_name=='HARVEY'), NAME_COLS+['medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school
1316,HARVEY,JOEL,COHEN,DUKE MEDICAL SCHOOL
1315,HARVEY,JAY,COHEN,SUNY DOWNSTATE MEDICAL CENTER COLLEGE OF MEDICINE


time: 25.9 ms


In [32]:
t.loc[t.clean_last_name=='COHEN', NAME_COLS+['first_name', 'sim']]

,clean_first_name,clean_middle_name,clean_last_name,first_name,sim
16,HARVEY,JAY,COHEN,Ronald,58.0
17,HARVEY,JAY,COHEN,Harvey,100.0
18,HARVEY,JAY,COHEN,Donald,65.0
53,MAX,HARRY,COHEN,Max,100.0
74,JEROLD,ARNOLD,COHEN,Jerold,100.0


time: 61.1 ms


In [33]:
apps4.loc[apps4.clean_last_name=='COHEN', NAME_COLS+['first_name', 'medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,first_name,medical_school
3657,STEPHEN,USHER,COHEN,Stephen,UNIVERSITY OF CHICAGO
3645,STEPHEN,MORRIS,COHEN,Stephen,JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE
3591,STEPHEN,ALAN,COHEN,Stephen,GEORGETOWN UNIVERSITY SCHOOL OF MEDICINE
3580,STANLEY,N,COHEN,Stanley,UNIVERSITY OF PENNSYLVANIA SCHOOL OF MEDICINE
3438,RONALD,JAY,COHEN,Ronald,JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE
3218,ROBERT,HOWARD,COHEN,Robert,NEW YORK UNIVERSITY COLLEGE OF MEDICINE
2873,PHILIP,LAWRENCE,COHEN,Philip,YALE UNIVERSITY SCHOOL OF MEDICINE
2888,PHILIP,NaN,COHEN,Philip,HARVARD MEDICAL SCHOOL
2459,MAX,HARRY,COHEN,Max,HARVARD MEDICAL SCHOOL
2411,MARTIN,H,COHEN,Martin,NEW YORK UNIVERSITY COLLEGE OF MEDICINE


time: 72.2 ms


In [34]:
um = man[~man.dno.isin(t_1.dno)]

um.shape

(0, 8)

time: 38.2 ms


In [35]:
um

,dno,clean_first_name,clean_middle_name,clean_last_name,application_year_min,application_year_max,dno_medical_school,medical_school


time: 61.4 ms


In [36]:
# t2 = pd.merge(left=apps4, right=um, on=['clean_last_name', 'clean_first_name'], how='inner', suffixes=['_x', '_y'])
# print t2.shape
# print um.shape

# t2['sim'] = t2[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

# t2['clean_middle_name'] = t2['clean_middle_name_y']

# t2_1 = t2.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')
# t2_1.shape

time: 50.7 ms


In [37]:
t3 = t_1[NAME_COLS+['dno', PERSON_ID]]
# t3 = pd.concat([t2_1[NAME_COLS+['dno', PERSON_ID]], t_1[NAME_COLS+['dno', PERSON_ID]]], axis=0)

time: 61 ms


In [38]:
man_dno = pd.merge(left=apps4, right=t3, on=PERSON_ID, how='left', suffixes=['_x', '_y'])
print man_dno.shape
print t3.shape
print man.shape

(4115, 90)
(103, 5)
(104, 8)
time: 78.7 ms


In [39]:
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno = man_dno.rename(columns={'clean_last_name_x': 'clean_last_name', 'clean_first_name_x': 'clean_first_name', 
                       'clean_middle_name_x': 'clean_middle_name'})
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno.loc[mask, 'clean_last_name'] = man_dno.loc[mask, 'clean_last_name_y']
mask = ~pd.isnull(man_dno.clean_first_name_y)
man_dno.loc[mask, 'clean_first_name'] = man_dno.loc[mask, 'clean_first_name_y']
mask = ~pd.isnull(man_dno.clean_middle_name_y)
man_dno.loc[mask, 'clean_middle_name'] = man_dno.loc[mask, 'clean_middle_name_y']

time: 55.5 ms


In [40]:
apps5 = man_dno.loc[pd.isnull(man_dno['dno']), :].drop(['dno', 'clean_middle_name_y', 'clean_first_name_y', 'clean_last_name_y'], axis=1) 
has_dno = man_dno.loc[~pd.isnull(man_dno['dno']), NAME_COLS+['dno', 'person_uuid']] 
# apps5 = man_dno.drop(['dno', 'clean_middle_name_y', 'clean_first_name_y', 'clean_last_name_y'], axis=1) 

time: 50.6 ms


In [41]:
# mark femalse
NIH['is_female'] = 0
female_mask = (NIH.clean_first_name.isin(FEMALE_FIRST_NAMES))  
NIH.loc[female_mask, 'is_female'] = 1


apps5['is_female'] = 0
female_mask = (apps5.clean_first_name.isin(FEMALE_FIRST_NAMES))  
apps5.loc[female_mask, 'is_female'] = 1

time: 156 ms


In [42]:
def get_first_letter(str_var):
    if pd.isnull(str_var) or str_var=='':
        return np.nan
    return str_var[0]

time: 14.9 ms


In [43]:
NIH['clean_first_initial'] = NIH.clean_first_name.apply(get_first_letter)
NIH['clean_middle_initial'] = NIH.clean_middle_name.apply(get_first_letter)

time: 65.3 ms


In [44]:
print NIH.shape
print  apps5.shape

(4045, 33)
(4012, 86)
time: 56.9 ms


In [45]:
# After cleaning apps2 to match cleaning in Clean NIH Applicant notebook, we try to start merging
sims_cols = ['medical_school_sim', 'clean_middle_name_sim', 'clean_first_name_sim']

time: 46.9 ms


In [46]:
NIH['fuzzy_merge_col'] = NIH[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
apps5['fuzzy_merge_col'] = apps5[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
match1 = df_get_closest_matches(apps5, NIH, 'fuzzy_merge_col', suffixes=['_x', '_y']) 
print match1.shape

(4045, 119)
time: 5min 35s


In [47]:
# add last name counter to each
app_counter = Counter(apps4.clean_last_name.values)
NIH_counter = Counter(NIH.clean_last_name.values)
match1['last_name_counts_x'] = match1.clean_last_name_x.apply(lambda x: app_counter[x])
match1['last_name_counts_y'] = match1.clean_last_name_y.apply(lambda x: NIH_counter[x])


time: 24.4 ms


In [48]:
def check_match(row):
    # address and application year match
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if row['clean_last_name_sim'] < 90 or app_eod_year_diff > 8:
        return 0
    if not pd.isnull(row['medical_school_sim']) and row['medical_school_sim'] < 90:
        # drop people with no medical school similarity
        return 0
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 80:
        return 0
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 90:
        return 1
    if row['last_name_counts_x'] < 2 and row['last_name_counts_y'] < 2:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 5):
        return 1
    # if matching application year and med schools match
    if (app_eod_year_diff < 5) and row['medical_school_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 5) and row['clean_first_name_sim'] > 80:
        return 1
    return 0


feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'clean_last_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

match2 = add_similarity_features(match1, feature_dict, check_match, suffixes=['_x', '_y'])

time: 2.75 s


In [49]:
# match2.loc[(match2.is_match==2), sims_cols+['is_match', 'clean_first_name_x', 'clean_first_name_y',
#                         'clean_last_name_x', 'clean_last_name_y','medical_school_x', 'medical_school_y', PERSON_ID, NIH_ID]]

time: 10.6 ms


In [50]:
# select out people who match and make sure each person id and dno only 1x in data set
match3 = match2[match2.is_match==1].sort_values(['clean_last_name_x']+sims_cols, ascending=False).dropna(
    axis=0, subset=[RAW_CARD_ID])
print match3.shape

(2435, 126)
time: 86.7 ms


In [51]:
def filter_one_match_per_group(df, dedupe_col, sim_cols):
    # to merge cols should be a dict the names of the extra cols to merge in
    # values should be col names to rename
    # sim cols should be name of the columns to use as features
    # sim mask should be mask that accounts as actual mask
    # dedupe col is name of col to dedupe on

    def count_matches(id_list_arr):
        # for each id, make sure matched on 1x in data set
        # should be applied with rolling apply so takes in a dataframe and must return single value
        # unpack already matched ids from string
        current_id1 = id_list_arr[-1]
        other_matches = id_list_arr[:-1]
        is_dup = np.any(other_matches[:] == current_id1)
        if is_dup:
            return True
        return False

    # for each uuid, check for duplicates and choose best match based on sim cols
    # order of the sim cols should be with most important first
    dup_flag = '{}_duplicate'.format(dedupe_col)
    df[dup_flag] = 0
    df.loc[:, dup_flag] = df[
        dedupe_col].expanding(center=False, min_periods=0).apply(func=count_matches)

    df_matches = df[df['is_match'] == 1].sort_values([dedupe_col] + sim_cols, ascending=False)
    return df_matches.drop_duplicates([dedupe_col], keep='first')


time: 39.4 ms


In [52]:
match4 = filter_one_match_per_group(match3, 'raw_uuid', sims_cols)
print match4.shape

(2428, 127)
time: 138 ms


In [53]:
match5 = filter_one_match_per_group(match4, NIH_ID, sims_cols)
print match5.shape

(2428, 128)
time: 86.3 ms


In [54]:
# get nonmatched NIH people and not matched applicants 
nm_apps = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match5[RAW_CARD_ID].dropna().values)
nm_NIH = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match5[NIH_ID].dropna().values)

(1583, 87)
(1617, 34)
time: 29.3 ms


In [55]:
# do another round of matching just on last name
nm_match1 = df_get_closest_matches(nm_apps, nm_NIH, 'clean_last_name', suffixes=['_x', '_y']) 
print nm_match1.shape


(2811, 119)
time: 13.9 s


In [56]:
def check_match(row):
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if app_eod_year_diff > 8:
        return 0
    
    if not pd.isnull(row['medical_school_sim']) and row['medical_school_sim'] < 80:
        return 0
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    max_name_sim = max(row['clean_first_name_sim'], row['medical_school_sim'], row['clean_middle_name_sim'])
    if max_name_sim < 80:
        return 0
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 90:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 6):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 70:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 6) and row['medical_school_sim'] > 90:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_first_name_sim'] > 90:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_middle_name_sim'] > 90:
        return 1
    return 0

time: 24.6 ms


In [57]:
nm_feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

nm_match2 = add_similarity_features(nm_match1, nm_feature_dict, check_match, suffixes=['_x', '_y'])


time: 756 ms


In [58]:
nm_match3 = nm_match2.loc[(nm_match2.is_match==1) & (nm_match2.index!='MORTON'), :].reset_index(
    drop=False).rename(columns={'index': 'clean_last_name'})

time: 15.9 ms


In [59]:
print nm_match3.shape

(105, 124)
time: 11.1 ms


In [60]:
# merge the manual people with dno with the dno data set and append to the matches list
man_dno2 = man_dno.loc[~pd.isnull(man_dno['dno']), :].drop(['clean_first_name_y', 'clean_middle_name_y',
                                                            'clean_last_name_y'], axis=1)
man_dno2.loc[:, 'dno'] = man_dno2.dno.astype(int)
NIH.loc[:, 'dno'] = NIH.dno.astype(int)

time: 26.1 ms


In [61]:
man_dno2.loc[man_dno2.clean_last_name=='COHEN', NAME_COLS+['first_name']]

,clean_first_name,clean_middle_name,clean_last_name,first_name
1613,MAX,HARRY,COHEN,Max
2388,JEROLD,ARNOLD,COHEN,Jerold
2752,HARVEY,JAY,COHEN,Harvey


time: 27.5 ms


In [62]:
print man_dno2.shape
man_dno_merge = pd.merge(left=man_dno2, right=NIH, on='dno', how='inner')
print man_dno_merge.shape

(103, 87)
(103, 120)
time: 28.3 ms


In [63]:
# stewart and sherwin have eod years outside range
man_dno2.loc[~man_dno.dno.isin(man_dno_merge.dno), NAME_COLS+['dno']]

,clean_first_name,clean_middle_name,clean_last_name,dno


time: 57 ms


In [64]:
man_dno_merge2 = man_dno_merge.rename(columns={'clean_first_name_x': 'clean_first_name2', 
                                               'clean_middle_name_x': 'clean_middle_name2',
                             'clean_last_name_x': 'clean_last_name2'})


time: 15.2 ms


In [65]:
man_dno_merge2.loc[man_dno_merge2.clean_last_name_y=='COHEN', NAME_COLS+['first_name']]

,clean_first_name,clean_middle_name,clean_last_name,first_name
46,NaN,NaN,NaN,Max
64,NaN,NaN,NaN,Jerold
77,NaN,NaN,NaN,Harvey


time: 29.6 ms


In [66]:
fails = man_dno2.loc[~man_dno2.dno.isin(man_dno_merge.dno), :]
print fails.shape

(0, 87)
time: 53.2 ms


In [67]:
# append matches together
match6 = pd.concat([nm_match3, match5, man_dno_merge2], axis=0)

time: 80.5 ms


In [68]:
# get nonmatched NIH people and not matched applicants 
nm_apps2 = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match6[RAW_CARD_ID].dropna().values)
nm_NIH2 = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match6[NIH_ID].dropna().values)

(1479, 87)
(1411, 34)
time: 21 ms


In [69]:
nm_NIH3 = nm_NIH2.loc[
    (nm_NIH2.eod_year< 1976) & (
        nm_NIH2.eod_year>1963), NAME_COLS+['medical_school', 'eod_year']].sort_values('clean_last_name')
print nm_NIH3.shape

(149, 5)
time: 16.8 ms


In [70]:
apps_match = nm_apps2.loc[nm_apps2.clean_last_name.isin(nm_NIH3.clean_last_name.values)]
test_merge = pd.merge(left=nm_NIH3, right=apps_match, on='clean_last_name', how='inner').sort_values('clean_last_name')
test_merge = test_merge[sorted(test_merge.columns)]
if OUTPUT_CSV:
    test_merge.to_csv(os.path.join(CORRECTIONS_DIR, 'test_merge_missing_NIH.csv'), index=False)

time: 25.7 ms


In [71]:
match6.loc[pd.isnull(match6.clean_last_name), 'clean_last_name'] =  match6.loc[
    pd.isnull(match6.clean_last_name), 'clean_last_name_x']
to_drop = [c for c in match6.columns if c.endswith('_sim') or '_counts' in c]
match7= match6.drop(to_drop+[
        'dup_flag', 'eod_year_diff', 'fuzzy_merge_col_x', 'fuzzy_merge_col_y', 'unknown',
        'Unnamed: 0', 'raw_uuid_duplicate', 'dno_duplicate', 'count_missing',
                'clean_last_name_x', 'clean_last_name_y', 'is_match'], axis=1)

time: 25.3 ms


In [72]:
# consolidate columns in match6
match7a = consolidate_merge_cols(match7, ['_x', '_y'], [])

['citizenship', 'clean_first_initial', 'clean_first_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'intern_dates', 'internship_hospital', 'is_female', 'medical_school', 'res_dates', 'residency_hospital', 'residency']
time: 13.3 s


In [73]:
mask = ~pd.isnull(match7a.clean_first_name2)
match7a.loc[mask, 'clean_first_name'] = match7a.loc[mask, 'clean_first_name2']

mask = ~pd.isnull(match7a.clean_last_name2)
match7a.loc[mask, 'clean_last_name'] = match7a.loc[mask, 'clean_last_name2']

mask = ~pd.isnull(match7a.clean_middle_name2)
match7a.loc[mask, 'clean_middle_name'] = match7a.loc[mask, 'clean_middle_name2']

time: 19.7 ms


In [74]:
match8 = pd.concat([
        man_dno[~pd.isnull(man_dno['dno'])], match7a, apps4.loc[
            ~apps4[PERSON_ID].isin(match7a[PERSON_ID].values),:]], axis=0).drop(['clean_first_name2', 
                                                                                'clean_middle_name2', 
                                                                                'clean_last_name2'], axis=1)
print match8.shape

(4219, 104)
time: 61.6 ms


In [75]:
# replace date of birth with dob whenever date of birth missing and dob is not
match8.loc[
    (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'date_of_birth'] = match8.loc[
        (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'dob']


c1 = 'date_of_birth'
c2 = 'dob'
match8.loc[(pd.isnull(match8[c1])) & (~pd.isnull(match8[c2])), [c1, c2]]

,date_of_birth,dob


time: 23.2 ms


In [76]:
match8.loc[(match8.clean_last_name=='KNOWLER') & (match8.clean_first_name=='JAN'), 'dno'] = 1922
match8.loc[
    (match8.clean_last_name== 'E ROSS HARVARD') & (match8.clean_first_name=='MICHAEL'), 'clean_last_name'] = 'ROSS'


time: 17.7 ms


In [77]:
match9 = match8.drop(['dob'], axis=1).reset_index(drop=True).sort_values(
    NAME_COLS+['dno']).drop_duplicates(NAME_COLS+['dno']).drop_duplicates(NAME_COLS+[PERSON_ID])
# print sorted(match9.columns)
print match8.shape
print match9.shape

(4219, 104)
(4115, 103)
time: 44.2 ms


In [78]:
# check for dno duplicates 
dups_dno = match9.loc[(~pd.isnull(match9.dno)) & (
        match9.duplicated('dno', keep=False)), NAME_COLS+['dno', PERSON_ID, 'medical_school']]

dups_merge = pd.merge(
    left=dups_dno, right=NIH.loc[NIH.dno.isin(dups_dno.dno), NAME_COLS+['dno', 'medical_school']], on=['dno'], how='left')


time: 20.2 ms


In [79]:
def get_sim_score(row):
    sim_cols = NAME_COLS + ['medical_school']
    sims = []
    for col in sim_cols:
        sim_1 = get_name_str_sim(row[['{}_x'.format(col), '{}_y'.format(col)]])
        sims.append(sim_1)
    return np.mean(sims)

time: 12.4 ms


In [80]:
dups_merge['sim'] = dups_merge.apply(get_sim_score, axis=1)

time: 25.9 ms


In [81]:
dups_merge = dups_merge.sort_values(['dno', 'sim'], ascending=False).rename(columns={'dno':'old_dno'})
dups_merge['dno'] = 0 

time: 16.1 ms


In [82]:
best_match = dups_merge.groupby('old_dno').first().person_uuid.values
print len(best_match)

2
time: 15.5 ms


In [83]:
mask = (dups_merge.sim > 94) & (dups_merge.person_uuid.isin(best_match))
print sum(mask)
dups_merge.loc[mask, 'dno'] = dups_merge[mask]['old_dno']
# match9.loc[(~pd.isnull(match9.clean_first_name_y)), ['clean_first_name', 'clean_first_name_y']]

2
time: 81.5 ms


In [84]:
# dups_merge.loc[dups_merge.dno==dups_merge.old_dno]

time: 57.8 ms


In [85]:
# reset duplicates to null
match9.loc[match9.person_uuid.isin(dups_merge.loc[dups_merge['dno']==0, PERSON_ID]), 'dno'] = np.nan

match9.loc[match9.person_uuid==35, NAME_COLS+[PERSON_ID, 'dno']]

match9.loc[(match9.duplicated('dno', keep=False) & (~pd.isnull(match9.dno))), NAME_COLS+['dno', PERSON_ID]]

,clean_first_name,clean_middle_name,clean_last_name,dno,person_uuid


time: 89.2 ms


In [86]:
# need to check we haven't created amy frankenstein matches
# ie. people who shouldn't really match together
# get a
match9.loc[(~pd.isnull(match9.first_name) & (
    match9.clean_first_name!=match9.first_name.str.upper())), NAME_COLS+['first_name', 'middle_name', 'last_name', 'med_school', 'original_medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,first_name,middle_name,last_name,med_school,original_medical_school
2605,ALBERT,LELAND,ALBRIGHT,Adelbert,Leland,"Albright, Jr.",Louisiana State University School of Medicine,Louisiana State University School of Medicine ...
825,ARTHUR,P,RICHARDSON,Arthur,P.,"Richardson, Jr.",University of Florida,University of Florida College of Medicine
95,ASHLEY,THOMSON,HAASE,Ashley,Thomson,Haase,NaN,Columbia University College of Physicians and ...
4087,BENJAMIN,DARYLL,CAHAN,Leslie,Daryll,Cahan,NaN,UCLA School of Medicine
4083,BERNARD,S,ARON,Ber,NaN,Aron,NaN,New York University School of Medicine
4049,CATHERINE,E,CLEMENTS,Janice,E.,Clements,NaN,University of Maryland School of Medicine
2541,CHARLES,MILLER,BAGLEY,Charles,Miller,"Bagley, Jr.",Harvard Medical School,Harvard Medical School
3924,DEAN,T,MASON,Dean,T.,Mason,NaN,Duke University School of Medicine
1629,ELBERT,CARMACK,HOLMES,E,Carmack,Holmes,University of North Carolina,University of North Carolina at Chapel Hill Sc...
1418,FRANCIS,JOHN,KROLIKOWSKI,Fraocis /Francis,John,Krolikowski,Tufts,Tufts University School of Medicine


time: 85.2 ms


In [87]:
match9.is_female.describe()

count    4115.000000
mean        0.008991
std         0.094408
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: is_female, dtype: float64

time: 16.1 ms


In [88]:
# add an international applicant flag
# also drop anyone missing first, middle names and med school
match9['is_foreign'] = 0
match9.loc[:, 'is_foreign'] = match9.medical_school.apply(is_foreign_med_school)

time: 2.56 s


In [89]:
match9.is_foreign.describe()

count    4115.000000
mean        0.001944
std         0.044055
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: is_foreign, dtype: float64

time: 15 ms


In [90]:
match9['control_flag'] = 0
match9.loc[pd.isnull(match9.dno) & pd.isnull(match9.year_accepted), 'control_flag'] = 1

time: 18.1 ms


In [91]:
match9.loc[pd.isnull(match9.application_year), 'application_year'] = match9.loc[
    pd.isnull(match9.application_year), 'application_year_1']

time: 13.4 ms


In [92]:
def total_number_applications(app_years):
    unique_apps_years = app_years.dropna().unique()
    return unique_apps_years.shape[0]

match9['number_applications'] = match9[
    ['application_year', 'application_year_1', 'application_year_2', 'application_year_3']].apply(
        total_number_applications, axis=1)

time: 1.65 s


In [93]:
def applications_max_min(app_years, fnc):
    unique_apps_years = app_years.dropna().unique()
    if unique_apps_years.shape[0] == 0:
        print app_years
        return np.nan
    return fnc(unique_apps_years)

time: 11.8 ms


In [94]:
match9['application_year_max'] = match9[
    ['application_year', 'application_year_1', 'application_year_2', 'application_year_3']].apply(
        funcy.rpartial(applications_max_min, max), axis=1)


time: 1.56 s


In [95]:
match9['application_year_min'] = match9[
    ['application_year', 'application_year_1', 'application_year_2', 'application_year_3']].apply(
        funcy.rpartial(applications_max_min, min), axis=1)


time: 1.57 s


In [96]:
match9['time_period_flag'] = 0
match9.loc[(match9.application_year_max>1964) & (match9.application_year_max<1976), 'time_period_flag'] = 1

time: 17.4 ms


In [97]:
# match9A.loc[(match9A.clean_last_name=='MCCHURE') & (pd.isnull(match9A.clean_first_name))]]
match9.loc[match9.clean_last_name=='MCCLURE MCCHURE', 'clean_last_name'] = 'MCCLURE'
match9.loc[match9.clean_last_name=='PERPICH', 'application_year'] = 1967
match9.loc[match9.clean_last_name=='PERPICH', 'application_year_min'] = 1967
match9.loc[match9.clean_last_name=='PERPICH', 'application_year_max'] = 1967
mathch9 = match9[match9.clean_last_name!='BRADEN R']
match9_1 = match9[~((match9.clean_last_name=='BULKEY') & (match9.clean_first_name=='GREGORY'))]
match9_2 = match9_1[~((match9_1.clean_last_name=='KNOWLER') & (match9_1.clean_first_name=='JAN'))]
match9_3 = match9_2[~((match9_2.clean_last_name=='COLLIN') & (match9_2.clean_first_name=='ROBERT'))]
match9_4 = match9_3[~((match9_3.clean_last_name=='BULLARD') & (match9_3.clean_first_name=='BRIAN'))]
# CHESEBRE, COLDBERG, Robert Collin, DIEZMAN, GLASSROBTH, HUGH HAYWOOD, Bart Kentover, jan knowler, robert jeffery kramer
# SAIRAI, william sullivan

time: 46.7 ms


In [98]:
# fix some 
match9_4.loc[(
        match9_4.clean_first_name=='WILLIAM') & (match9_4.clean_middle_name=='WILLIAM') & 
             (match9_4.clean_last_name=='SULLIVAN'), 'clean_middle_name'] = np.nan

match9_4.loc[match9_4.clean_middle_name=='JEFFERY', 'clean_middle_name'] = 'JEFFREY'
match9_4.loc[match9_4.clean_last_name=='GLASSROBTH', 'clean_last_name'] = 'GLASSROTH'

time: 105 ms


/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [124]:
# name corrections
match9_4.loc[((match9_4.clean_first_name=='ALBERT') & (match9_4.first_name=='Adelbert')), 'clean_first_name'] = 'ADELBERT'
match9_4.loc[((match9_4.clean_first_name=='BENJAMIN') & (match9_4.clean_last_name=='CAHAN')), 'clean_first_name'] = 'LESLIE'
match9_4.loc[((match9_4.clean_first_name=='GEORGES')), 'clean_first_name'] = 'GEORGE'
match9_4.loc[((match9_4.clean_first_name=='JACKS')), 'clean_first_name'] = 'JACK'
match9_4.loc[((match9_4.clean_first_name=='HAVERY')), 'clean_first_name'] = 'HARVEY'
match9_4.loc[((match9_4.clean_first_name=='LAWRENCE') & (match9_4.first_name=='Laurence')), 'clean_first_name'] = 'LAURENCE'
match9_4.loc[((match9_4.clean_first_name=='PHILLIP') & (match9_4.first_name=='Philip')), 'clean_first_name'] = 'PHILIP'
match9_4.loc[((match9_4.clean_first_name=='FREDERIC') & (match9_4.clean_last_name=='MUSHINSKI')), 'clean_middle_name'] = 'COSTEP'
# name corrections
match9_4.loc[((match9_4.clean_first_name=='ALBERT') & (match9_4.first_name=='Adelbert')), 'clean_first_name'] = 'ADELBERT'
match9_4.loc[((match9_4.clean_first_name=='BENJAMIN') & (match9_4.clean_last_name=='CAHAN')), 'clean_first_name'] = 'LESLIE'
match9_4.loc[((match9_4.clean_first_name=='GEORGES')), 'clean_first_name'] = 'GEORGE'
match9_4.loc[((match9_4.clean_first_name=='JACKS')), 'clean_first_name'] = 'JACK'
match9_4.loc[((match9_4.clean_first_name=='HAVERY')), 'clean_first_name'] = 'HARVEY'
match9_4.loc[((match9_4.clean_first_name=='LAWRENCE') & (match9_4.first_name=='Laurence')), 'clean_first_name'] = 'LAURENCE'
match9_4.loc[((match9_4.clean_first_name=='PHILLIP') & (match9_4.first_name=='Philip')), 'clean_first_name'] = 'PHILIP'
match9_4.loc[((match9_4.clean_first_name=='FREDERIC') & (match9_4.clean_last_name=='MUSHINSKI')), 'clean_middle_name'] = 'COSTEP'


time: 386 ms


In [125]:
test = ['PERPICH',  'BRADEN', 'BRADEN R', 'BULKEY', 'BULKLEY', 
       'CHESEBRE', 'COLDBERG', 'COLLIN', 'DIEZMAN', 'DIETZMAN', 'GLASSROBTH', 
        'GLASSROTH', 'HAYWARD', 'HAYWOOD', 'MCCLURE MCCHURE', 'MCCLURE', 'MCCHURE',
       'KETOVER', 'KENTOVER', 'KNOWLER', 'KRAMER', 'SARAI', 'SARAL', 'SARAL', 'SULLIVAN', 'COLLINS', 
       'KOEHLER']
match9_4.loc[match9_4.clean_last_name.isin(test), NAME_COLS+['medical_school']].sort_values('clean_last_name')

,clean_first_name,clean_middle_name,clean_last_name,medical_school
2388,WILLIAM,D,BRADEN,HARVARD MEDICAL SCHOOL
2333,BERNADINE,HEALY,BULKLEY,HARVARD MEDICAL SCHOOL
2332,GREGORY,BARTLETT,BULKLEY,HARVARD MEDICAL SCHOOL
117,JAMES,E,COLDBERG,HARVARD MEDICAL SCHOOL
2212,ROBERT,CHAPLIN,COLLINS,JOAN & SANFORD I. WEILL MEDICAL COLLEGE CORNEL...
2211,STEVEN,HAMES,COLLINS,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...
123,DALE,E,DIETZMAN,UNIVERSITY OF VERMONT COLLEGE OF MEDICINE
3605,JEFFREY,LEONARD,GLASSROTH,UNIVERSITY OF CINCINNATI COLLEGE OF MEDICINE
1699,HUGH,R,HAYWARD,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE
4092,BART,PETER,KENTOVER,JOAN SANFORD I WEILL MEDICAL COLLEGE CORNELL U...


time: 59.3 ms


In [126]:
# delete people missing first and middle names
print match9_4.shape
match9A = match9_4.loc[~((pd.isnull(match9_4.clean_first_name) & pd.isnull(match9_4.clean_middle_name))), :]
match9A2 = match9A.loc[~(
        (match9A.medical_school=='UNIVERSITY OF MINNESOTA MEDICAL SCHOOL DULUTH') & (
            match9A.clean_last_name=='PERPICH')), :]
match9A3 = match9A2.loc[~((pd.isnull(match9A2.clean_first_name) & (match9A2.clean_last_name=='MCCHURE'))), :]
match9A4 = match9A3.loc[~(
        (match9A3.clean_first_name==match9A3.clean_middle_name) & (match9A3.clean_last_name=='BROWER')), :]
match9B = match9A4.loc[~pd.isnull(match9A4.application_year_min), :]
print match9A.shape
print match9B.shape

(4111, 109)
(4109, 109)
(4108, 109)
time: 55 ms


In [127]:
match9A2.loc[pd.isnull(match9A2.application_year_min), NAME_COLS+['application_year_min', 'medical_school', 'application_year']]
match9B.loc[match9B.clean_last_name=='PERPICH', NAME_COLS+['application_year_min', 'medical_school', 'application_year']]

,clean_first_name,clean_middle_name,clean_last_name,application_year_min,medical_school,application_year


time: 39.6 ms


In [128]:
mask = match9B.clean_first_name.apply(has_suffix)
match9B.loc[mask, 'clean_suffix'] = match9B.loc[mask, 'clean_first_name'].apply(get_suffix)

match9B.loc[mask, 'clean_first_name'] = 'SPENCER'

time: 35.6 ms


In [129]:
# check for person id duplicates
dups_mask = match9B.duplicated(PERSON_ID, keep=False)
match9B.loc[dups_mask, NAME_COLS+['medical_school', PERSON_ID, 'address']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school,person_uuid,address


time: 17.9 ms


In [130]:
import hashlib
from random import randint
max(match9B.person_uuid.values)

4152.0

time: 13.8 ms


In [131]:
max_id = max(match9B.person_uuid.values)
print max_id
dups_mask = match9B.duplicated(PERSON_ID, keep=False)
dups = match9B.ix[dups_mask, NAME_COLS+[PERSON_ID, 'medical_school', 'address']]
dups['new_id'] = dups.person_uuid.apply(lambda x: max_id+randint(10, 1000))

4152.0
time: 27.3 ms


In [132]:
match9B.loc[match9B.duplicated(PERSON_ID, keep=False), NAME_COLS+[PERSON_ID, 'dno']]


,clean_first_name,clean_middle_name,clean_last_name,person_uuid,dno


time: 24.4 ms


In [133]:
match9C = pd.merge(left=match9B, right=dups, on=NAME_COLS+['medical_school', 'address', PERSON_ID], how='left')

time: 45.7 ms


In [134]:
mask = ~pd.isnull(match9C.new_id)

time: 17.3 ms


In [135]:
match9C.loc[match9C.person_uuid==3800,NAME_COLS+[PERSON_ID, 'new_id']]

,clean_first_name,clean_middle_name,clean_last_name,person_uuid,new_id
3813,THOMAS,E,HOFFMAN,3800.0,NaN


time: 27.5 ms


In [136]:
match9C.loc[mask, PERSON_ID] = match9C[mask]['new_id'] 

time: 24.3 ms


In [137]:
del match9C['new_id']

time: 11.8 ms


In [138]:
dups_mask = match9C.duplicated(PERSON_ID, keep=False)
match9C.loc[dups_mask, NAME_COLS+['medical_school', PERSON_ID, 'address']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school,person_uuid,address


time: 19.7 ms


In [139]:
match9C.rename(columns={'res_dates': 'residency_dates', 'intern_dates': 'internship_dates', 
                      'clean_college_trans': 'clean_college'}, inplace=True)

IMPORTANT_COLS = [NIH_ID, PERSON_ID, 'application_year_min', 'application_year_max', 'eod_year',
                  'clean_first_name', 'clean_middle_name', 
                 'clean_last_name', 'control_flag', 'time_period_flag', 'year_accepted',
                  'rejected', 'rejection_date', 'clean_college', 'medical_school',
                'residency_dates', 'internship_dates', 'is_female', 'is_foreign', 'number_applications']

other_cols = sorted([i for i in match9C.columns if i not in IMPORTANT_COLS])

# order columns so important ones are 
match10 = match9C[IMPORTANT_COLS+other_cols].sort_values(['clean_last_name', 'clean_first_name', 'application_year_max']).drop(
    ['clean_first_name_y', 'clean_middle_name_y', 'clean_last_name_y'], axis=1)

match11 = match10.dropna(subset=[PERSON_ID], axis=0).sort_values(['clean_last_name', 'clean_first_name'])

mask = (pd.isnull(match11.eod_year) & ~pd.isnull(match11.dno))

match11['is_female'] = 0

match11.loc[match11.clean_first_name.isin(FEMALE_FIRST_NAMES), 'is_female'] = 1

match11.loc[mask, 'eod_year'] = match11[mask].dno.apply(lambda x: NIH.loc[NIH.dno==x, 'eod_year'].values[0])
# wide_apps5.to_pickle(os.path.join(APP_DATA_DIR, 'all_apps_plus_NIH_info.p'))
match11.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info.csv'), index=False)

time: 1.13 s


In [140]:
match9A.loc[match9A.clean_last_name.isin(['HARRIN']), NAME_COLS+[PERSON_ID, 'medical_school', 'application_year', 'dno', 'residency','residency_hospital',
                                                            'internship_hospital', 'residency_dates']]

,clean_first_name,clean_middle_name,clean_last_name,person_uuid,medical_school,application_year,dno,residency,residency_hospital,internship_hospital,residency_dates
4208,NaN,SHELOR,HARRIN,4121.0,JOAN SANFORD I WEILL MEDICAL COLLEGE CORNELL U...,1970.0,NaN,OPHTHALMOLOGY,JOHNS HOPKINS HEALTH SYSTEM,UNIVERSITY OF WASHINGTON,NaN


time: 29.6 ms


In [141]:
# NIH.loc[NIH.dno.isin(dups_dno.dno), NAME_COLS+['dno']]

time: 11.7 ms


In [142]:
dups_dno = match11.loc[(~pd.isnull(match11.dno)) & (match11.duplicated('dno', keep=False)), NAME_COLS+['dno', PERSON_ID, 'medical_school']]
print dups_dno.shape
dups_merge = pd.merge(
    left=dups_dno, right=NIH.loc[NIH.dno.isin(dups_dno.dno), NAME_COLS+['dno', 'medical_school']], on=['dno'], how='left')

dups_merge

(0, 6)


,clean_first_name_x,clean_middle_name_x,clean_last_name_x,person_uuid,medical_school_x,clean_first_name_y,clean_middle_name_y,clean_last_name_y,dno,medical_school_y


time: 33.9 ms


In [143]:
# check for uuid dups
match11.loc[match11.duplicated(PERSON_ID, keep=False), NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name


time: 23.1 ms
